# **Dependancies**

In [201]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import *
from keras.callbacks import *
import os
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from commons import mean_absolute_percentage_error
from keras.layers import *
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.losses import LogCosh

# **Loading Data**

In [161]:
PATH_TO_DATA = "reg_interval1.csv"
data = pd.read_csv(PATH_TO_DATA)

In [162]:
data.head()

,difficulty14ema,difficulty30rsi,difficulty3sma,difficulty7ema,difficulty7wma,difficulty90ema,difficulty90trx,difficulty90wma,hashrate30var,mining_profitability,mining_profitability90mom,transactionfees90rsiUSD,priceUSD
0,1838321625272,94.687,1889597185141,1880883535862,1885474257661,1469637581968,0.489,1555222335970,1.034456e+36,2.174,1.106,55.636,736695
1,1850696935834,94.804,1908940844897,1893446765519,1898215033558,1479780414331,0.491,1566085496216,9.827391e+35,2.431,1.595,55.721,736696
2,1861422204988,94.804,1928284504652,1902869187761,1908883274481,1489700327301,0.493,1576751520156,9.450696e+35,2.737,1.659,56.381,736697
3,1870717438255,94.804,1931136454488,1909936004443,1917478980430,1499402220207,0.494,1587220407791,1.029070e+36,2.606,1.456,55.861,736698
4,1878773307086,94.804,1931136454488,1915236116954,1924002151405,1508890884696,0.496,1597492159121,1.048469e+36,2.631,1.560,57.594,736699


In [163]:
data.shape

(1187, 13)

In [164]:
X = data.iloc[:,:-1]

In [165]:
y=data.iloc[:,-1:]

In [166]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.2, train_size=0.8, shuffle=True, random_state=7)

In [167]:
X_train.shape

(949, 12)

In [168]:
estimators=[]

In [169]:
estimators.append(['robust',RobustScaler()])

In [170]:
estimators.append(['mixmax',MinMaxScaler()])

In [171]:
scale=Pipeline(estimators,verbose=True)

In [172]:
X_train=scale.fit_transform(X_train)

[Pipeline] ............ (step 1 of 2) Processing robust, total=   0.0s
[Pipeline] ............ (step 2 of 2) Processing mixmax, total=   0.0s


In [173]:
X_test=scale.transform(X_test)

In [174]:
X_train=np.reshape(X_train,(X_train.shape[0],1,X_train.shape[1]))

In [175]:
X_test=np.reshape(X_test,(X_test.shape[0],1,X_test.shape[1]))

In [176]:
y_train=y_train.values

In [177]:
y_train=np.reshape(y_train, (y_train.shape[0],1,1))

In [178]:
y_test=y_test.values

In [179]:
y_test=np.reshape(y_test,(y_test.shape[0],1,1))

In [180]:
X_train.shape

(949, 1, 12)

In [181]:
y_train.shape

(949, 1, 1)

In [182]:
X_test.shape

(238, 1, 12)

# **Model Architecture + Training**

In [183]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [184]:

adam = optimizers.Adam(learning_rate=lr_schedule(0), amsgrad=True)

Learning rate:  0.001


In [185]:
model = Sequential()
model.add(Bidirectional(LSTM(350, return_sequences=True, activation='relu'), input_shape=(1, X_train.shape[2])))
model.add(Bidirectional(LSTM(350, return_sequences=True, activation='relu')))
model.add(Dense(1))
model.compile(loss="log_cosh", optimizer=adam, metrics=['mae'])

E:\DS440proj\BTCpred\.venv\lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [199]:
mcp_save = ModelCheckpoint('model/LSTM_reg_seven_new.keras', save_best_only=True, monitor='val_loss', mode='auto')
earlyStopping = EarlyStopping(monitor='val_loss', patience=100, verbose=1, mode='auto')

ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=model/LSTM_reg_seven_new.h5

In [187]:
#model.compile(optimizer='adam', loss='log_cosh', metrics=['accuracy'])
model.fit(X_train,y_train, epochs=5000, batch_size=32, validation_data=(X_test,y_test), callbacks=[mcp_save,earlyStopping])

Epoch 1/5000
30/30 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - loss: 737291.8125 - mae: 737292.5625 - val_loss: 737273.1875 - val_mae: 737273.9375
Epoch 2/5000
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 737229.0000 - mae: 737229.8125 - val_loss: 737043.7500 - val_mae: 737044.5000
Epoch 3/5000
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 736882.5625 - mae: 736883.2500 - val_loss: 735970.9375 - val_mae: 735971.6875
Epoch 4/5000
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 735349.1875 - mae: 735349.9375 - val_loss: 732666.1250 - val_mae: 732666.7500
Epoch 5/5000
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 731047.2500 - mae: 731047.9375 - val_loss: 725184.0625 - val_mae: 725184.8750
Epoch 6/5000
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 722474.4375 - mae: 722475.0625 - val_loss: 711317.0000 - val_mae: 711317.6250
Epoch 7/5000
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 706264.8750 - mae: 706265.5000 - val_loss: 689074.8750 - val_mae: 689075.6250
Epoch 8/5000
30/30 ━━━━━━━━

# **Testing Model**

In [200]:
from tensorflow.keras.models import load_model
prediction_model = load_model('model/LSTM_reg_seven_new.keras',compile=False)

ValueError: A total of 2 objects could not be loaded. Example error message for object <LSTMCell name=lstm_cell, built=True>:

Layer 'lstm_cell' expected 3 variables, but received 0 variables during loading. Expected: ['kernel', 'recurrent_kernel', 'bias']

List of objects that could not be loaded:
[<LSTMCell name=lstm_cell, built=True>, <LSTMCell name=lstm_cell, built=True>]

In [202]:
y_pred = np.ravel(model.predict(X_test))

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


In [203]:
y_test=np.ravel(y_test)

In [204]:
r2=r2_score(y_test,y_pred) #testing score/ r^2
r2

0.41578356830179086

In [205]:
mae=mean_absolute_error(y_test,y_pred) #mae
mae

199.23844537815125

In [206]:
rmse=np.sqrt(mean_squared_error(y_test,y_pred)) #rmse
rmse

260.4584156800888

In [207]:
mape=mean_absolute_percentage_error(y_test,y_pred) #mape
mape

0.027017613857229183

In [208]:
pd.DataFrame(zip(['MAE','RMSE','MAPE','R^2'],[mae,rmse,mape,r2])).transpose()

,0,1,2,3
0,MAE,RMSE,MAPE,R^2
1,199.238445,260.458416,0.027018,0.415784


In [209]:
pd.DataFrame([y_test,y_pred]).transpose()

,0,1
0,737692.0,737382.8125
1,736911.0,736915.3125
2,737646.0,737534.1250
3,737704.0,737383.6875
4,737435.0,737227.0000
...,...,...
233,736958.0,736906.8125
234,737067.0,736717.1875
235,737850.0,737338.3750
236,737572.0,737454.7500
